# ParaView Test
This notebook is useful as an extremely minimal, "hello world"-type example. It creates a synthetic data source and computes an isosurface mesh from it. Then, it creates a ParaView display widget showing the primary render view. The notebook further demonstrates how the same view can be displayed multiple times, and how multiple RenderViews can be handled.

In [ ]:
import paraview
from paraview.simple import *

# create the primary 'Render View'
renV = CreateView('RenderView')
renV.ViewSize = [800, 500]
renV.CameraPosition = [50, 0, 0] # Initial camera settings
renV.CameraViewUp = [0, 1.0, 0]  #

# create a new 'Wavelet'
wavelet = Wavelet()

# create a new 'Contour' filter to compute an isosurface from our volume
contour = Contour(Input=wavelet)
contour.ContourBy = ['POINTS', 'RTData']
contour.Isosurfaces = [157.0909652709961]
contour.UpdatePipeline()

# create the visual representation object for the contour
contourDisplay = Show(contour, renV)
contourDisplay.Representation = 'Surface'
contourDisplay.AmbientColor = contourDisplay.DiffuseColor = [0.286, 0.851, 0.114]

# import the PVDisplay widget, then instantiate one for renV
from ipyparaview.widgets import PVDisplay
disp = PVDisplay(renV)

# show the widget once
display(disp)

In [ ]:
# show the same widget again
display(disp)

In [ ]:
# create a second 'Render View'
renV2 = CreateView('RenderView')
renV2.ViewSize = [800, 500]
renV2.CameraPosition = [50, 0, 0] # Initial camera settings
renV2.CameraViewUp = [0, 1.0, 0]  #

# create a new 'Contour' filter to compute an isosurface from our volume
contour2 = Contour(Input=wavelet)
contour2.ContourBy = ['POINTS', 'RTData']
contour2.Isosurfaces = [50, 75, 100, 125]
contour2.UpdatePipeline()

# create the visual representation object for the contour
contour2Display = Show(contour2, renV2)
contour2Display.Representation = 'Surface'
contour2Display.AmbientColor = contourDisplay.DiffuseColor = [0.286, 0.851, 0.114]

disp2 = PVDisplay(renV2)
display(disp2)